Chose a new test file since previous sequences were lost to multimapping when aligned to full PalmDB index with kb:

In [2]:
fastq = "/home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2.fastq.gz"
test_fastq = fastq

outfolder = "/home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark"

### Generate test file

In [5]:
# # Create new file keeping only first X reads
# !zcat $fastq | head -400000000 > $test_fastq

# # Copy and unzip complete original fastq
# !cp $fastq /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/raw_short/
# !gunzip /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/raw_short/SRR12698503_2.fastq.gz
# !rm /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/raw_short/SRR12698503_2.fastq.gz


gzip: stdout: Broken pipe


In [6]:
# Print number of lines (reads = lines / 4) in test_fastq
!wc -l $test_fastq

400000000 /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/raw_short/v2/SRR12698539_2.fastq


### Run Kraken2

In [7]:
!kraken2-build

Must select a task option.
Usage: kraken2-build [task option] [options]

Task options (exactly one must be selected):
  --download-taxonomy        Download NCBI taxonomic information
  --download-library TYPE    Download partial library
                             (TYPE = one of "archaea", "bacteria", "plasmid",
                             "viral", "human", "fungi", "plant", "protozoa",
                             "nr", "nt", "UniVec", "UniVec_Core")
  --special TYPE             Download and build a special database
                             (TYPE = one of "greengenes", "silva", "rdp")
  --add-to-library FILE      Add FILE to library
  --build                    Create DB from library
                             (requires taxonomy d/l'ed and at least one file
                             in library)
  --clean                    Remove unneeded files from a built database
  --standard                 Download and build default database
  --help                     Print this mess

Tried to use palmdb with kraken-build --protein argument, but it needs NCBI taxonomy IDs in the fasta:

In [122]:
# kraken_palmdb = "/home/laura/projects/kraken2-2.1.2/kraken_palmdb"
# virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/mmseqs_covmode_1/palmdb_DB_clu_rep.fasta"

# !cd /home/laura/projects/kraken2-2.1.2 \
#     && kraken2-build --db kraken_palmdb --download-taxonomy

# !cd /home/laura/projects/kraken2-2.1.2 \
#     && kraken2-build \
#         --db kraken_palmdb \
#         --kmer-len 31 \
#         --minimizer-len 31 \
#         --protein \
#         --no-masking \
#         --add-to-library $virus_fasta

scan_fasta_file.pl: unable to determine taxonomy ID for sequence u5665


krakendb: taxonomy and viral library from NCBI + manually added Ebola genome (otherwise EBOV is not detected) (also see: kraken_bash.sh file)

In [12]:
# k2protocol_db = "/home/laura/projects/kraken2-2.1.2/k2protocol_db"
krakendb = "/home/laura/projects/kraken2-2.1.2/krakendb"

In [13]:
!kraken2

Need to specify input filenames!
Usage: kraken2 [options] <filename(s)>

Options:
  --db NAME               Name for Kraken 2 DB
                          (default: none)
  --threads NUM           Number of threads (default: 1)
  --quick                 Quick operation (use first hit or hits)
  --unclassified-out FILENAME
                          Print unclassified sequences to filename
  --classified-out FILENAME
                          Print classified sequences to filename
  --output FILENAME       Print output to filename (default: stdout); "-" will
                          suppress normal output
  --confidence FLOAT      Confidence score threshold (default: 0.0); must be
                          in [0, 1].
  --minimum-base-quality NUM
                          Minimum base quality used in classification (def: 0,
                          only effective with FASTQ input).
  --report FILENAME       Print a report with aggregrate counts/clade to file
  --use-mpa-style         Wi

In [14]:
!/usr/bin/time -v \
    ~/bin/kraken2 \
        --db $krakendb \
        --threads 32 \
        --minimum-hit-groups 3 \
        --report-minimizer-data \
        --report $outfolder/kraken/SRR12698503.k2report \
        $test_fastq > $outfolder/kraken/SRR12698503.kraken2

Loading database information... done.
460911662 sequences (40560.23 Mbp) processed in 1108.602s (24945.6 Kseq/m, 2195.21 Mbp/m).. bp) ...25103084952 bp) ...26746195256 bp) ...29028006744 bp) ...29218586936 bp) ...35924949368 bp) ... bp) ...
  3101998 sequences classified (0.67%)
  457809664 sequences unclassified (99.33%)
	Command being timed: "/home/laura/bin/kraken2 --db /home/laura/projects/kraken2-2.1.2/krakendb --threads 32 --minimum-hit-groups 3 --report-minimizer-data --report /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503.k2report /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2.fastq.gz"
	User time (seconds): 6144.41
	System time (seconds): 328.04
	Percent of CPU this job got: 583%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 18:30.05
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 1437552


### View kraken report in Pavian
1. Upload SRR12698503.kreport2 files to [Pavian viewer](https://fbreitwieser.shinyapps.io/pavian/)  
2. Go to 'Comparison' tab
3. Select 'species', 'reads', '%', 'rank', 'clade' and 'taxon'
4. Search for 'viruses'
5. Select 'CSV' to download csv overview

### Extract Kraken reads
The extract_kraken_reads.py script (from the KrakenTools GitHub repo) extracts reads that matched a particular species, identified by the taxonomy ID that is provided with the -t parameter:

In [15]:
ebov_tid = 186538

In [16]:
# # Download script
# !curl -O https://raw.githubusercontent.com/jenniferlu717/KrakenTools/master/extract_kraken_reads.py

!/usr/bin/time -v \
    python extract_kraken_reads.py \
    -k $outfolder/kraken/SRR12698503.kraken2 \
    --include-children \
    -s $test_fastq \
    -t $ebov_tid \
    -r $outfolder/kraken/SRR12698503.k2report \
    -o $outfolder/kraken/SRR12698503_EBOV.tid10298.1.fa

PROGRAM START TIME: 06-25-2023 02:23:42
>> STEP 0: PARSING REPORT FILE /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503.k2report
	1 taxonomy IDs to parse
>> STEP 1: PARSING KRAKEN FILE FOR READIDS /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503.kraken2
	460.91 million reads processed
	102140 read IDs saved
>> STEP 2: READING SEQUENCE FILES AND WRITING READS
	102140 read IDs found (460.91 mill reads processed)
	102140 reads printed to file
	Generated file: /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503_EBOV.tid10298.1.fa
PROGRAM END TIME: 06-25-2023 03:44:00
	Command being timed: "python extract_kraken_reads.py -k /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503.kraken2 --include-children -s /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698539_2.fastq.gz -t 186538 -r /home/laura/projects/virus-watch-data/benchmarking/ebola_be

### Align extracted reads to the species genomes using Bowtie2

Ebola genome ViralProj14703 (linked to NC_002549.1) downloaded from https://www.ncbi.nlm.nih.gov/data-hub/genome/?taxon=186538

Generate Bowtie2 genome index:

In [1]:
b_index = "/home/laura/projects/kraken2-2.1.2/b_index"

In [18]:
# !/usr/bin/time \
#     bowtie2-build \
#     /home/laura/projects/kraken2-2.1.2/ebov_fna/ncbi_dataset/data/GCA_000848505.1/GCA_000848505.1_ViralProj14703_genomic.fna \
#     $b_index/ebov

Align extracted reads to species genome:

In [4]:
!/usr/bin/time -v \
    bowtie2 \
    -x $b_index/ebov \
    -f -p 8 \
    -U $outfolder/kraken/SRR12698503_EBOV.tid10298.1.fa \
    -S $outfolder/kraken/SRR12698503_EBOV_aligned.sam

102140 reads; of these:
  102140 (100.00%) were unpaired; of these:
    18607 (18.22%) aligned 0 times
    83533 (81.78%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
81.78% overall alignment rate
	Command being timed: "bowtie2 -x /home/laura/projects/kraken2-2.1.2/b_index/ebov -f -p 8 -U /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503_EBOV.tid10298.1.fa -S /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503_EBOV_aligned.sam"
	User time (seconds): 4.99
	System time (seconds): 0.59
	Percent of CPU this job got: 643%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.86
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 110092
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 30342
	Voluntary context switches: 34628

### Use SAMtools to convert the SAM files to sorted BAM files

In [5]:
!/usr/bin/time -v \
    samtools view \
    -bS -F4 $outfolder/kraken/SRR12698503_EBOV_aligned.sam \
    > $outfolder/kraken/SRR12698503_EBOV_aligned.bam

	Command being timed: "samtools view -bS -F4 /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503_EBOV_aligned.sam"
	User time (seconds): 0.84
	System time (seconds): 0.06
	Percent of CPU this job got: 91%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.99
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 2844
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 9562
	Voluntary context switches: 80
	Involuntary context switches: 1
	Swaps: 0
	File system inputs: 0
	File system outputs: 7280
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


In [6]:
!/usr/bin/time -v \
    samtools sort \
    $outfolder/kraken/SRR12698503_EBOV_aligned.bam \
    -o $outfolder/kraken/SRR12698503_EBOV_sorted.bam

	Command being timed: "samtools sort /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503_EBOV_aligned.bam -o /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503_EBOV_sorted.bam"
	User time (seconds): 0.54
	System time (seconds): 0.02
	Percent of CPU this job got: 90%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.63
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 29036
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 2112
	Voluntary context switches: 10
	Involuntary context switches: 2
	Swaps: 0
	File system inputs: 0
	File system outputs: 3160
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


In [7]:
!/usr/bin/time -v \
    samtools index \
    $outfolder/kraken/SRR12698503_EBOV_sorted.bam

	Command being timed: "samtools index /home/laura/projects/virus-watch-data/benchmarking/ebola_benchmark/kraken/SRR12698503_EBOV_sorted.bam"
	User time (seconds): 0.09
	System time (seconds): 0.00
	Percent of CPU this job got: 92%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:00.10
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 2544
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 735
	Voluntary context switches: 3
	Involuntary context switches: 1
	Swaps: 0
	File system inputs: 0
	File system outputs: 8
	Socket messages sent: 0
	Socket messages received: 0
	Signals delivered: 0
	Page size (bytes): 4096
	Exit status: 0


### Visualize alignment using Pavian
Upload the sorted BAM file and the index of the sorted BAM file (.bam and .bai files) to [Pavian’s Alignment Viewer](https://fbreitwieser.shinyapps.io/pavian/).